## Setup

This notebook uses Python. Run cells in order from top to bottom.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from tqdm.notebook import tqdm
tqdm.pandas()

## Configuration

Edit the values in the cell below to match your data **before running the notebook**.

In [ ]:
# =============================================================
# CONFIGURATION — Edit these values before running
# =============================================================

# Path to directory containing .txt files (one document per file)
DATA_DIR = "data/"

# --- Required: at least one text column ---
TEXT_COL_PRIMARY   = "text"   # Content of each .txt file
TEXT_COL_SECONDARY = None     # No secondary column for plain text files

# --- Optional metadata columns ---
# Set any of these to None if your data doesn't include them
DATE_COL   = None  # No date column
DOMAIN_COL = None  # No domain column
URL_COL    = None  # No URL column

# --- Deduplication ---
# Columns to deduplicate on — set to [] to skip deduplication entirely
DEDUPE_COLS = []

# --- Output ---
OUTPUT_DIR = "output"   # Folder to save results (created automatically if it doesn't exist)

## Load Data & Remove Duplicates 🧹

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Option A: Load from a CSV file ---
# df = pd.read_csv("data/your_file.csv")
# print(f"Loaded {len(df):,} rows from CSV")

# --- Option B: Load data from .txt files — one document per file ---
records = []
for fname in sorted(os.listdir(DATA_DIR)):
    if fname.endswith(".txt"):
        fpath = os.path.join(DATA_DIR, fname)
        with open(fpath, "r", encoding="utf-8") as f:
            content = f.read().strip()
        records.append({"filename": os.path.splitext(fname)[0], "text": content})


df = pd.DataFrame(records)
print(f"Loaded {len(df):,} documents from '{DATA_DIR}'")
df.head()

Rows that will be removed as duplicates:

In [ ]:
if DEDUPE_COLS:
    dupes = df[df.duplicated(subset=DEDUPE_COLS, keep=False)].sort_values(by=DEDUPE_COLS)
    print(f"{len(dupes):,} duplicate rows found (columns used: {DEDUPE_COLS})")
    display(dupes.head())
else:
    print("DEDUPE_COLS is empty — skipping deduplication preview.")

In [ ]:
if DEDUPE_COLS:
    before = len(df)
    df = df.drop_duplicates(subset=DEDUPE_COLS, keep="last").reset_index(drop=True)
    print(f"Removed {before - len(df):,} duplicates. {len(df):,} rows remaining.")
df.head()

## Embeddings

Generate a vector embedding for each row. **Run Option A *or* Option B — not both.**

| | Option A | Option B |
|---|---|---|
| **Model** | `all-MiniLM-L6-v2` (local) | `text-embedding-3-small` (OpenAI) |
| **Cost** | Free | ~$0.02 / million tokens |
| **Requires** | Nothing extra | API key in `.env` |
| **Download** | ~80 MB (first run only) | None |
| **Quality** | Good enough for clustering | Higher |

In [ ]:
import tiktoken

ENCODING   = "cl100k_base"
MAX_TOKENS = 8000
enc        = tiktoken.get_encoding(ENCODING)

def make_combined_text(row):
    """Combine primary and (optional) secondary text columns for embedding."""
    primary = str(row[TEXT_COL_PRIMARY]) if pd.notna(row[TEXT_COL_PRIMARY]) else ""
    if (TEXT_COL_SECONDARY
            and TEXT_COL_SECONDARY in row.index
            and pd.notna(row[TEXT_COL_SECONDARY])):
        secondary = str(row[TEXT_COL_SECONDARY])
        return f"Title: {primary}; Content: {secondary}"
    return primary

df["combined"] = df.apply(make_combined_text, axis=1)
df["n_tokens"] = df["combined"].apply(lambda x: len(enc.encode(x)))
df = df.sort_values("n_tokens", ascending=False)
print(f"Longest article: {df['n_tokens'].max():,} tokens | Average: {df['n_tokens'].mean():.0f} tokens")
df[["combined", "n_tokens"]].head()

In [ ]:
too_long = df[df["n_tokens"] > MAX_TOKENS]
print(f"Removing {len(too_long):,} articles exceeding {MAX_TOKENS:,} tokens.")
df = df[df["n_tokens"] <= MAX_TOKENS].reset_index(drop=True)
print(f"{len(df):,} rows remaining.")

### Option A — Local model

No API key needed. Model downloads ~80 MB on first run, then works offline.

> Skip to Option B if you prefer to use the OpenAI API.

In [ ]:
# # ── Option A: local model ──────────────────────────────────────────
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("all-MiniLM-L6-v2")  # downloads ~80 MB on first run

# df["embedding"] = model.encode(
#     df["combined"].tolist(),
#     show_progress_bar=True,
#     batch_size=64,
# ).tolist()

# print("Done.")

### Option B — OpenAI API

Higher quality embeddings. Requires an API key in `.env` and the `openai` + `python-dotenv` packages.

> Skip this cell if you already ran Option A above.

In [ ]:
# ── Option B only ──────────────────────────────────────────────────
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

EMBEDDING_MODEL = "text-embedding-3-small"

# ── Option B: OpenAI embeddings ────────────────────────────────────
def get_embeddings(texts):
    response = client.embeddings.create(model=EMBEDDING_MODEL, input=texts)
    return [item.embedding for item in response.data]

def process_in_batches(df, column, batch_size=30):
    all_embeddings = []
    texts = df[column].tolist()
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i : i + batch_size]
        all_embeddings.extend(get_embeddings(batch))
    return all_embeddings

df["embedding"] = process_in_batches(df, "combined")

df = df.drop(columns=["combined", "n_tokens"])
df.head()

## Dimensionality Reduction (t-SNE)

Embeddings are high-dimensional vectors (~1,536 dimensions). t-SNE projects them to 2D (x, y) for plotting.
This step is cached — re-running the notebook will load saved coordinates instead of recomputing.

In [ ]:
from sklearn.manifold import TSNE

tsne_cache = os.path.join(OUTPUT_DIR, "tsne-cache.csv")

if os.path.exists(tsne_cache):
    print(f"Loading cached t-SNE coordinates from '{tsne_cache}'")
    coords = pd.read_csv(tsne_cache)
    df["x"] = coords["x"].values
    df["y"] = coords["y"].values
else:
    print("Running t-SNE (this may take a few minutes on large datasets)...")
    matrix = np.array(df["embedding"].tolist())
    tsne = TSNE(
        n_components=2,
        perplexity=min(30, len(df) // 5),  # scales with dataset size
        random_state=42,
        init="pca",           # stable, deterministic
        learning_rate="auto") # adapts to dataset size
    vis_dims = tsne.fit_transform(matrix)
    df["x"] = vis_dims[:, 0]
    df["y"] = vis_dims[:, 1]
    df[["x", "y"]].to_csv(tsne_cache, index=False)
    print(f"Done. Coordinates cached to '{tsne_cache}'")

In [ ]:
df.head()

## Topic Modeling (DBSCAN)

[DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) clusters points in vector space — articles in the same cluster tend to be about the same topic.

- **Topic 0** = noise (articles that don't fit into any cluster)
- Adjust `DBSCAN_EPS` and `DBSCAN_MIN_SAMPLES` in the Configuration cell if you get too few or too many clusters.

In [ ]:
# --- Clustering parameters (cosine distance, range 0–2) ---
# eps:         max cosine distance between two points in the same cluster
#              lower = tighter clusters (e.g. 0.2); higher = looser (e.g. 0.5)
# min_samples: minimum number of documents required to form a cluster
DBSCAN_EPS         = 0.5
DBSCAN_MIN_SAMPLES = 2

from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances

df = df.reset_index(drop=True)

matrix = np.array(df["embedding"].tolist())

# Diagnostic: show distance distribution to help tune eps
dists = cosine_distances(matrix)
np.fill_diagonal(dists, np.nan)
flat = dists[~np.isnan(dists)]
print("Cosine distance percentiles (use these to set DBSCAN_EPS):")
for p in [10, 25, 50, 75, 90]:
    print(f"  {p}th percentile: {np.percentile(flat, p):.3f}")

labels = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES, metric="cosine").fit_predict(matrix)
df["topic"] = labels + 1   # shift so DBSCAN noise (-1) becomes topic 0

topic_counts = (df.groupby("topic")
                  .size()
                  .reset_index(name="count")
                  .sort_values("count", ascending=False))
n_clusters = (topic_counts["topic"] > 0).sum()
n_noise    = topic_counts.loc[topic_counts["topic"] == 0, "count"].sum()
print(f"\nFound {n_clusters} clusters + {n_noise:,} uncategorized (noise) articles")

df["topic_label"] = df["topic"].apply(
    lambda t: "Uncategorized" if t == 0 else f"Topic {t}"
)

summary = (df.groupby(["topic", "topic_label"])
             .size()
             .reset_index(name="count")
             .sort_values("count", ascending=False))
summary

## Export

In [ ]:
# --- Full dataset (all columns, including embedding vectors) ---
full_path = os.path.join(OUTPUT_DIR, "data-with-embeddings.csv")
df.to_csv(full_path, index=False)
print(f"Full dataset    → '{full_path}'")

# --- Visualization dataset ---
# Renames columns to match the visualization tool's expected schema:
#   title, text, date, org, x, y, url
rename_map = {}
if TEXT_COL_PRIMARY:   rename_map[TEXT_COL_PRIMARY]   = "title"
if TEXT_COL_SECONDARY: rename_map[TEXT_COL_SECONDARY] = "text"
if DATE_COL:           rename_map[DATE_COL]           = "date"
if DOMAIN_COL:         rename_map[DOMAIN_COL]         = "org"
if URL_COL:            rename_map[URL_COL]            = "url"

vis_df = df.rename(columns=rename_map)

vis_cols = [c for c in ["title", "text", "date", "org", "x", "y", "url", "topic_label"]
            if c in vis_df.columns]

vis_path = os.path.join(OUTPUT_DIR, "data-for-visualization.csv")
vis_df[vis_cols].to_csv(vis_path, index=False)
print(f"Visualization   → '{vis_path}'")
print(f"\nColumns exported: {vis_cols}")

## Interactive Chart

In [ ]:
import plotly.express as px
import textwrap

def wrap_hover(text, width=60):
    if pd.isna(text):
        return ""
    lines = textwrap.wrap(str(text), width=width)
    return "<br>".join(lines)

plot_df = df.copy()
plot_df["_primary"] = plot_df[TEXT_COL_PRIMARY].apply(lambda t: wrap_hover(t, width=60))

hover_name_col = "filename" if "filename" in df.columns else TEXT_COL_PRIMARY
hover_data = {"_primary": True, "x": False, "y": False}
if TEXT_COL_SECONDARY and TEXT_COL_SECONDARY in df.columns:
    plot_df["_secondary"] = plot_df[TEXT_COL_SECONDARY].apply(lambda t: wrap_hover(t, width=60))
    hover_data["_secondary"] = True

fig = px.scatter(
    plot_df,
    x="x",
    y="y",
    color="topic_label",
    hover_name=hover_name_col,
    hover_data=hover_data,
    title="Semantic Map",
    width=900,
    height=700,
)
fig.update_traces(marker=dict(size=9, opacity=0.7))
fig.update_layout(
    legend_title_text="Topic",
    hoverlabel=dict(namelength=-1),
)
fig.show()